# 从0️⃣跑通中文大模型构建｜🚀Day002

## Tokenizer选择

**tokenization**是大模型训练的第一步，是将文本转换为模型可以理解的数字表示（后面也能反向decode回来）。

其中目前比较主流的是[BPE(Byte Pair Encoding)](https://zhuanlan.zhihu.com/p/424631681)（详细的介绍可以参考链接文章，下面只进行一些简单的介绍）。

**BPE**是一种简单的数据压缩形式，这种方法用数据中不存在的一个字节表示最常出现的连续字节数据。这样的替换需要重建全部原始数据。

### BPE简介

假设我们要编码如下数据

> aaabdaaabac

字节对“aa”出现次数最多，所以我们用数据中没有出现的字节“Z”替换“aa”得到替换表

> Z <- aa

数据转变为

> ZabdZabac

在这个数据中，字节对“Za”出现的次数最多，我们用另外一个字节“Y”来替换它（这种情况下由于所有的“Z”都将被替换，所以也可以用“Z”来替换“Za”），得到替换表以及数据

> Z <- aa
> Y <- Za

> YbdYbac

我们再次替换最常出现的字节对得到：

> Z <- aa
> Y <- Za
> X <- Yb

> XdXac

由于不再有重复出现的字节对，所以这个数据不能再被进一步压缩。

解压的时候，就是按照相反的顺序执行替换过程。




### 以ChatGLM3-6B的tokenizer为例

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True)

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


In [13]:
tokenizer("这是一个测试")

{'input_ids': [64790, 64792, 30910, 36037, 32882], 'attention_mask': [1, 1, 1, 1, 1], 'position_ids': [0, 1, 2, 3, 4]}

In [14]:
tokenizer.decode(tokenizer("这是一个测试")["input_ids"])

'[gMASK] sop 这是一个测试'

In [15]:
tokenizer.vocab_size

64798

### 选择ChatGLM3-6B的tokenizer的原因

该词表大小为64798，值得注意的是：这是一个很妙的数字，因为它**刚好在uint16的表示范围（0～65535的无符号整数）**，每一个token只需要两个字节即可表示。

当我们的语料较大时候，相比常用的int32可以**节省一半的存储空间**。

另外这里选择一个小尺寸的词表还有一个更重要的原因：我们后面的模型会选择一个小参数量的，如果词表过大，会导致**大部分参数被embedding层占用**，而无法训练出更好的模型。